In [1]:
import re
import pandas as pd
from datetime import datetime, timedelta,date
import numpy as np
import json
import os
from calendar import monthrange
import pymysql
import sys 
import requests
import matplotlib.pyplot as plt
from elasticsearch.helpers import scan
import seaborn as sns 
import datetime as dt
import xlsxwriter

In [2]:
df=pd.read_csv('C:/Users/PC/Downloads/googleplaystore.csv')
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [3]:
df.isna().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [4]:
df=df.fillna(0)
mean=df['Rating'].mean()
df['Rating']=df['Rating'].fillna(mean)
df['Rating']=df['Rating'].round(1)

In [5]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)

In [6]:
df.drop(df[df['Rating'] == 19.0].index, inplace = True)

In [7]:
df=df[df['Reviews'].str.isnumeric()]
df['Reviews']=df['Reviews'].astype('int64')

In [8]:
def size(size):
    if size == 'Varies with device':
        return 10
    elif size.endswith('M'):
        size = size[:-1]
        return float(size)
df['Size']=df['Size'].apply(size)

In [9]:
df['Installs']=df['Installs'].str.replace(r"[^a-zA-Z\d\_]+","")
df['Installs']=df['Installs'].astype('int64')

C:\Users\PC\AppData\Local\Temp\ipykernel_12540\1588397970.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Installs']=df['Installs'].str.replace(r"[^a-zA-Z\d\_]+","")


In [10]:
df['Price']=df['Price'].replace('[^0-9\.-]','',regex=True)
df['Price']=df['Price'].astype('float')
df.set_index('App')

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...
Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
Parkinson Exercices FR,MEDICAL,0.0,3,9.5,1000,Free,0.0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up


In [12]:
writer = pd.ExcelWriter('googleapp_clean.xlsx',engine = 'xlsxwriter')
df.to_excel(writer, sheet_name = 'dataset')
writer.save()
writer.close()

C:\Users\PC\AppData\Roaming\Python\Python310\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


#### Most Installed Apps 

In [14]:
df[['Installs','App','Category']].sort_values(by='Installs',ascending=False).head(10).reset_index(drop=True)

,Installs,App,Category
0,1000000000,Hangouts,COMMUNICATION
1,1000000000,YouTube,VIDEO_PLAYERS
2,1000000000,Gmail,COMMUNICATION
3,1000000000,Google Chrome: Fast & Secure,COMMUNICATION
4,1000000000,Subway Surfers,GAME
5,1000000000,Google Drive,PRODUCTIVITY
6,1000000000,Google Photos,PHOTOGRAPHY
7,1000000000,Google Play Games,ENTERTAINMENT
8,1000000000,Google Photos,PHOTOGRAPHY
9,1000000000,Google Chrome: Fast & Secure,COMMUNICATION


#### Most reviews Apps

In [15]:
df[['Reviews','App','Category']].sort_values(by='Reviews',ascending=False).head(10).reset_index(drop=True)

,Reviews,App,Category
0,78158306,Facebook,SOCIAL
1,78128208,Facebook,SOCIAL
2,69119316,WhatsApp Messenger,COMMUNICATION
3,69109672,WhatsApp Messenger,COMMUNICATION
4,66577446,Instagram,SOCIAL
5,66577313,Instagram,SOCIAL
6,66509917,Instagram,SOCIAL
7,56646578,Messenger – Text and Video Chat for Free,COMMUNICATION
8,56642847,Messenger – Text and Video Chat for Free,COMMUNICATION
9,44893888,Clash of Clans,GAME


#### Most Expansive Apps

In [16]:
df[['Price','App','Category']].sort_values(by='Price',ascending=False).head(10).reset_index(drop=True)

,Price,App,Category
0,400.00,I'm Rich - Trump Edition,LIFESTYLE
1,399.99,I AM RICH PRO PLUS,FINANCE
2,399.99,I'm Rich/Eu sou Rico/أنا غني/我很有錢,LIFESTYLE
3,399.99,💎 I'm rich,LIFESTYLE
4,399.99,I Am Rich Pro,FAMILY
5,399.99,most expensive app (H),FAMILY
6,399.99,I am rich(premium),FINANCE
7,399.99,I am rich,LIFESTYLE
8,399.99,I am Rich Plus,FAMILY
9,399.99,I am rich (Most expensive app),FINANCE


#### Most Rating Apps

In [18]:
df[['Rating','App','Category']].sort_values(by='Rating',ascending=False).head(10).reset_index(drop=True)

,Rating,App,Category
0,5.0,Noticias FC Barcelona,NEWS_AND_MAGAZINES
1,5.0,Nur təfsiri 1-ci cild,LIBRARIES_AND_DEMO
2,5.0,CJ Gospel Hour,SOCIAL
3,5.0,CJ'S TIRE AND AUTO INC.,PRODUCTIVITY
4,5.0,BG Guide,TRAVEL_AND_LOCAL
5,5.0,Story Time FD,FAMILY
6,5.0,"FD Calculator (EMI, SIP, RD & Loan Eligilibility)",FINANCE
7,5.0,Speeding Joyride & Car Meet App,DATING
8,5.0,Online Girls Chat Group,DATING
9,5.0,Comunidad De Fe Minitries,LIFESTYLE
